In [24]:
import os
import pandas as pd
import json
import re
from deep_translator import GoogleTranslator
import swifter

In [25]:
pd.options.display.max_colwidth = 500

In [26]:
dfs = []
for r, d, f in os.walk(os.getcwd()):
    for file in f:
        if 'withheldtweets.json' in file:
            dfs.append(pd.read_json("./censored_tweets/%s" % file, lines=True))

df_cen = pd.concat(dfs)
df_cen = df_cen.dropna(subset=['withheld_in_countries'])

In [27]:
df_cen.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'quoted_status_id', 'quoted_status_id_str',
       'quoted_status', 'quoted_status_permalink', 'is_quote_status',
       'extended_tweet', 'quote_count', 'reply_count', 'retweet_count',
       'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level',
       'lang', 'timestamp_ms', 'linked', 'display_text_range',
       'withheld_in_countries', 'extended_entities', 'possibly_sensitive',
       'retweeted_status', 'withheld_copyright'],
      dtype='object')

### create dataset with traduction

In [28]:
df_cen.set_index('id', inplace=True)

In [29]:
df_cen.shape

(41727, 38)

In [30]:
df_without_duplicate = df_cen.drop_duplicates("text")

In [31]:
df_without_duplicate.shape

(23081, 38)

In [33]:
def remove_user_mentions(tweets):
    return re.sub(r"@\w+", " ", tweets)

def remove_numbers(tweets):
	return re.sub(r"\d+([.,]\d+)?", " ", tweets)

def remove_URLs(tweets):   
    return re.sub(r'http\S+', '', tweets)

def remove_RT(tweets):
    return re.sub("RT  :", " ", tweets)

def remove_n(tweets):
    return tweets.replace("\n", "")

In [34]:
clean_tweets = df_without_duplicate["text"].apply(lambda x: remove_user_mentions(x))

In [35]:
clean_tweets = clean_tweets.apply(lambda x: remove_n(x))
clean_tweets = clean_tweets.apply(lambda x: remove_URLs(x))
clean_tweets = clean_tweets.apply(lambda x: remove_numbers(x))
clean_tweets = clean_tweets.apply(lambda x: remove_RT(x))

In [36]:
df_clean = df_without_duplicate.copy()

In [37]:
df_clean['text'] = clean_tweets

In [39]:
df_clean.shape

(23081, 38)

In [40]:
df_clean[clean_tweets.apply(lambda x: x.isspace())].shape

(530, 38)

In [41]:
df_clean

,created_at,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,...,filter_level,lang,timestamp_ms,linked,display_text_range,withheld_in_countries,extended_entities,possibly_sensitive,retweeted_status,withheld_copyright
id,,,,,,,,,,,,,,,,,,,,,
1365252415444946945,2021-02-26 10:48:35+00:00,1365252415444946944,#Balakot Pak Army is our pride ❤️👍,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,NaN,NaN,NaN,None,...,low,en,2021-02-26 10:48:35.662,no,"[0, 35]",[IN],"{'media': [{'id': 1365252409015033857, 'id_str': '1365252409015033857', 'indices': [36, 59], 'media_url': 'http://pbs.twimg.com/media/EvJZ1SQWYAEW5qu.jpg', 'media_url_https': 'https://pbs.twimg.com/media/EvJZ1SQWYAEW5qu.jpg', 'url': 'https://t.co/JirGPSMMGO', 'display_url': 'pic.twitter.com/JirGPSMMGO', 'expanded_url': 'https://twitter.com/hafizmirzaki/status/1365252415444946945/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 680, 'h': 636...",0.0,NaN,NaN
1365251727574900738,2021-02-26 10:45:51+00:00,1365251727574900736,لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ ہمارے فیس بک پیج پر لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔اس کے…,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,NaN,NaN,NaN,None,...,low,ur,2021-02-26 10:45:51.661,no,NaN,[IN],NaN,NaN,"{'created_at': 'Thu Feb 25 18:59:12 +0000 2021', 'id': 1365013492537978880, 'id_str': '1365013492537978880', 'text': 'لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ ہمارے فیس بک پیج پر 13 لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔ ا… https://t.co/ar04grdquW', 'display_text_range': [0, 140], 'source': '<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id...",NaN
1365249991137251328,2021-02-26 10:38:57+00:00,1365249991137251328,سید ابراہیم کا تعلق قبائلی ضلع کرم سے ہے۔ احساس بلا سود قرضہ لے کر انہوں نے اپنا سبزی کا کاروبار بڑھایا ہے جس سے ان کی…,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,NaN,NaN,NaN,None,...,low,ur,2021-02-26 10:38:57.662,no,NaN,[IN],NaN,NaN,"{'created_at': 'Fri Feb 26 05:23:00 +0000 2021', 'id': 1365170479468859396, 'id_str': '1365170479468859396', 'text': 'سید ابراہیم کا تعلق قبائلی ضلع کرم سے ہے۔ احساس بلا سود قرضہ لے کر انہوں نے اپنا سبزی کا کاروبار بڑھایا ہے جس سے… https://t.co/9afHbWpU1p', 'display_text_range': [0, 140], 'source': '<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_r...",NaN
1365242894362279938,2021-02-26 10:10:45+00:00,1365242894362279936,في مثل فجر هذا اليوم قبل عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. ستبقى #ذكرى_مجزرة_الحرم_الإ…,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",False,NaN,NaN,NaN,NaN,None,...,low,ar,2021-02-26 10:10:45.659,no,NaN,[IL],NaN,NaN,"{'created_at': 'Thu Feb 25 19:04:40 +0000 2021', 'id': 1365014870475866114, 'id_str': '1365014870475866114', 'text': 'في مثل فجر هذا اليوم قبل 27 عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. ستبقى… https://t.co/ZRweXMKZdW', 'display_text_range': [0, 140], 'source': '<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to...",NaN
1365242898569134080,2021-02-26 10:10:46+00:00,1365242898569134080,Mujhe aj bhi batting ki bari ni di😔 Yahin so jata hun! Subha jb sab ayenge pehle pehle me kheilunga🤔🤔😁,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,NaN,NaN,NaN,None,...,low,

In [42]:
df = df_clean.drop(df_clean[df_clean.text.apply(lambda x: x.isspace())].index)

In [43]:
df_clean[df_clean.text.apply(lambda x: x.isspace())].index

Int64Index([1365363136698679299, 1365461602166857733, 1365202687789305857,
            1365224900827389953, 1365219544663531520, 1365386226317025285,
            1365413761918504961, 1365848396699996163, 1365566690449387520,
            1365681815688798210,
            ...
            1361636627580416000, 1361659683677868040, 1361651186026360835,
            1361610450945802240, 1361837723506589698, 1361734002563043338,
            1361579048204263424, 1361663278204739593, 1361757016700690436,
            1361789837121105922],
           dtype='int64', name='id', length=530)

In [44]:
df_without_duplicate[df_clean.text.apply(lambda x: x.isspace())].to_csv("empty_text.csv")

In [45]:
translated = df_clean.text.swifter.apply(lambda x: GoogleTranslator(source='auto', target='en').translate(str(x)))

TooManyRequests: Server Error: You made too many requests to the server. According to google, you are allowed to make 5 requests per second and up to 200k requests per day. You can wait and try again later or you can try the translate_batch function

In [ ]:
df_clean['translated'] = translated

In [63]:
def remove_RT(tweets):
    return re.sub("RT  :", " ", tweets)

In [80]:
translator.translate("bonjour",lang_tgt='en') 

google_new_transError: 404 (Not Found) from TTS API. Probable cause: Unknown

In [81]:
from googletrans import Translator, constants

In [82]:
translator = Translator()

In [84]:
translator.translate('Hola Mundo')

AttributeError: 'NoneType' object has no attribute 'group'

In [85]:
from deep_translator import GoogleTranslator
translated = GoogleTranslator(source='auto', target='de').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig

In [ ]:
def (text):
    if te
    return ''.join([i for i in text if not i.isdigit()]

In [89]:
dd = df_cen["text"].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))

In [113]:
df_cen['extended_entities']

1      {'media': [{'id': 1365252409015033857, 'id_str': '1365252409015033857', 'indices': [36, 59], 'media_url': 'http://pbs.twimg.com/media/EvJZ1SQWYAEW5qu.jpg', 'media_url_https': 'https://pbs.twimg.com/media/EvJZ1SQWYAEW5qu.jpg', 'url': 'https://t.co/JirGPSMMGO', 'display_url': 'pic.twitter.com/JirGPSMMGO', 'expanded_url': 'https://twitter.com/hafizmirzaki/status/1365252415444946945/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 680, 'h': 636...
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [96]:
dd.size

41727

In [98]:
import swifter

In [115]:
aa = dd.head(12).apply(lambda x: GoogleTranslator(source='auto', target='en').translate(str(x)))

In [116]:
aa

1                                                                                                                                 #Balakot \nPak Army is our pride ❤️👍 https://t.co/JirGPSMMGO
2                                           RT @ZaidZamanHamid: Majority of people don't want to be guided.\nWe have lakhs of members on our Facebook page. 300,000 on our Twitter page.\nhis…
3                                           RT @ZaidZamanHamid: Majority of people don't want to be guided.\nWe have lakhs of members on our Facebook page. 300,000 on our Twitter page.\nhis…
4                              RT @SaniaNishtar: Syed Ibrahim belongs to tribal district Karam. By taking Ehsaas interest-free loan, he has expanded his vegetable business through which his…
5     RT @mosa_abumarzook: At the dawn of this day a year ago, the Ibrahimi Mosque presented a group of martyrs, as they knelt in prostration.\nThe #Memory_of_the_Haram_Massacre_will remain…
6     RT @mosa_abumarzook: At the dawn of thi

In [31]:
aa = dd.apply(lambda x: cleaning_URLs(x))

In [ ]:
def remove_hashtags(tweet):

In [64]:
def clean_tweets(df_text):
    clean_tweets = df_text.apply(lambda x: remove_URLs(x))
    clean_tweets = clean_tweets.apply(lambda x: remove_hashtags(x))
    clean_tweets = clean_tweets.apply(lambda x: remove_user_mentions(x))
    clean_tweets = clean_tweets.apply(lambda x: remove_numbers(x))
    clean_tweets = clean_tweets.apply(lambda x: remove_RT(x))
    return clean_tweets
    

In [65]:
clean_tweets(df_cen["text"]).head(10)

1                                                                                                      \nPak Army is our pride ❤️👍  
2          لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہمارے فیس بک پیج پر   لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔\nاس کے…
3          لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہمارے فیس بک پیج پر   لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔\nاس کے…
4          سید ابراہیم کا تعلق قبائلی ضلع کرم سے ہے۔ احساس بلا سود قرضہ  لے کر انہوں  نے اپنا سبزی کا کاروبار بڑھایا ہے جس سے ان کی…
5                                في مثل فجر هذا اليوم قبل   عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. \nستبقى  …
6                                في مثل فجر هذا اليوم قبل   عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. \nستبقى  …
7                         Mujhe aj bhi batting ki bari ni di😔\n Yahin so jata hun! Subha jb sab ayenge pehle pehle me kheilunga🤔🤔😁  
8                         Mujhe aj bhi batting ki bari ni di😔\n Yahin

In [34]:
df_cen["text"]

1                                                                                        #Balakot \nPak Army is our pride ❤️👍 https://t.co/JirGPSMMGO
2      RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہمارے فیس بک پیج پر 13 لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔\nاس کے…
3      RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہمارے فیس بک پیج پر 13 لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔\nاس کے…
4         RT @SaniaNishtar: سید ابراہیم کا تعلق قبائلی ضلع کرم سے ہے۔ احساس بلا سود قرضہ  لے کر انہوں  نے اپنا سبزی کا کاروبار بڑھایا ہے جس سے ان کی…
5       RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل 27 عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. \nستبقى #ذكرى_مجزرة_الحرم_الإ…
                                                                            ...                                                                      
636      RT @ZaidZamanHamid: یہ کوئی افریقہ کا ملک نہیں امریکی ریاست ٹیکساس ہے۔۔ شدید سردی اور برف ب

In [36]:
df_cen["text"].head(10)

1                                                                                       #Balakot \nPak Army is our pride ❤️👍 https://t.co/JirGPSMMGO
2     RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہمارے فیس بک پیج پر 13 لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔\nاس کے…
3     RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہمارے فیس بک پیج پر 13 لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔\nاس کے…
4        RT @SaniaNishtar: سید ابراہیم کا تعلق قبائلی ضلع کرم سے ہے۔ احساس بلا سود قرضہ  لے کر انہوں  نے اپنا سبزی کا کاروبار بڑھایا ہے جس سے ان کی…
5      RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل 27 عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. \nستبقى #ذكرى_مجزرة_الحرم_الإ…
6      RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل 27 عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. \nستبقى #ذكرى_مجزرة_الحرم_الإ…
7      RT @Saimhun: Mujhe aj bhi batting ki bari ni di😔\n Yahin so jata hun! Subha jb sab ayenge pehle peh